<a href="https://colab.research.google.com/github/SanjayMarreddi/ML_Internship_Technocolabs/blob/master/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [46]:
df=pd.read_csv('fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [47]:
#print(df.info())
print(df.describe())
print(df["Usage"].value_counts())


            emotion
count  15377.000000
mean       3.323145
std        1.876648
min        0.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        6.000000
Training    15376
Name: Usage, dtype: int64


In [48]:


X_train,y_train,X_test,y_test=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")



print(len(X_train),len(y_train))
print(len(X_test),len(y_test))


error occured at index :15376 and row:emotion                                                    6
pixels     254 224 105 83 94 97 109 112 111 111 114 114 1...
Usage                                                    NaN
Name: 15376, dtype: object
15376 15376
0 0


In [34]:

num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48


X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

y_train=np_utils.to_categorical(y_train, num_classes=num_labels)
y_test=np_utils.to_categorical(y_test, num_classes=num_labels)

#cannot produce



In [35]:
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

print(f"shape:{X_train.shape}")
print(f"shape:{X_test.shape}")

shape:(3372, 48, 48, 1)
shape:(0, 48, 48, 1)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [36]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

# Final Output Layer
model.add(Dense(num_labels, activation='softmax'))

# model.summary()


In [40]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=int(epochs/6),
          verbose=1,
          validation_data=(X_train[100:200], y_train[100:200]),
          shuffle=True)



Train on 3372 samples, validate on 100 samples
Epoch 1/5
3372/3372 [==============================] - 61s 18ms/step - loss: 1.8092 - accuracy: 0.2414 - val_loss: 1.7948 - val_accuracy: 0.2700
Epoch 2/5
3372/3372 [==============================] - 61s 18ms/step - loss: 1.7876 - accuracy: 0.2542 - val_loss: 1.7437 - val_accuracy: 0.2800
Epoch 3/5
3372/3372 [==============================] - 61s 18ms/step - loss: 1.7640 - accuracy: 0.2648 - val_loss: 1.7155 - val_accuracy: 0.2900
Epoch 4/5
3372/3372 [==============================] - 61s 18ms/step - loss: 1.7009 - accuracy: 0.3167 - val_loss: 1.6531 - val_accuracy: 0.3400
Epoch 5/5
3372/3372 [==============================] - 62s 18ms/step - loss: 1.6462 - accuracy: 0.3413 - val_loss: 1.5783 - val_accuracy: 0.3500


In [42]:

#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")